# Alphavec Example - Search

This notebook demonstrates running `alphavec.grid_search()` to perform a parameter grid search.

Install the optional dependencies to run this notebook: `pip install -r requirements.dev.txt`

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
from IPython.display import HTML, display

from alphavec import MarketData, SimConfig, grid_search, tearsheet, Metrics


## Load example market data

In [ ]:
data_dir = Path("./data")

weights = pd.read_feather(data_dir / "weights.feather")
close_prices = pd.read_feather(data_dir / "close_prices.feather")
open_prices = pd.read_feather(data_dir / "open_prices.feather")
funding_rates = pd.read_feather(data_dir / "funding_rates.feather")

display(weights.tail())
display(close_prices.tail())

symbols = weights.columns.tolist()
print("assets", len(symbols))
print("start", weights.index.min())
print("end", weights.index.max())


## Define a weight generator

`generate_weights(params)` can close over any data you want (here: `close_prices`).

This example uses a simple momentum signal and normalizes the weights to target a chosen gross leverage.

In [ ]:
def generate_weights(params):
    """Simple cross-sectional momentum signal from the sample market data.

    Notes:
    - Uses close-to-close momentum over `lookback`.
    - Uses an EWMA smoother for stability.
    - Shifts weights by 1 period to avoid lookahead.
    """

    lookback = int(params["lookback"])
    smooth_span = int(params["smooth_span"])
    leverage = float(params["leverage"])

    # Close-to-close momentum (more realistic than synthetic data).
    mom = close_prices.pct_change(lookback, fill_method=None)
    mom = mom.replace([np.inf, -np.inf], np.nan)

    if smooth_span > 1:
        mom = mom.ewm(span=smooth_span, adjust=False, min_periods=max(3, smooth_span // 5)).mean()

    # Cross-sectional z-score each day (only across tradable assets).
    tradable = close_prices.notna() & close_prices.shift(-1).notna() & open_prices.notna()
    x = mom.where(tradable)
    mu = x.mean(axis=1)
    sig = x.std(axis=1, ddof=1).replace(0.0, np.nan)
    z = x.sub(mu, axis=0).div(sig, axis=0)

    # Clip extreme z-scores and convert to weights.
    z = z.clip(-3.0, 3.0).fillna(0.0)
    denom = z.abs().sum(axis=1).replace(0.0, np.nan)
    w = z.div(denom, axis=0).fillna(0.0)

    # Avoid lookahead: signal at t is traded at t+1.
    w = w.shift(1).fillna(0.0)
    return w * leverage


base_params = {
    "lookback": 20,
    "smooth_span": 10,
    "leverage": 1.0,
}

weights_preview = generate_weights(base_params)
weights_preview.tail()


## Run a 2D grid search with metric objective

We search over `lookback x leverage` and `lookback x smooth_span`. The objective metric defaults to `Annualized Sharpe` (and must match the simulation metrics table index exactly).

In [ ]:
search_sets = [
    {"lookback": [5, 10, 20, 40, 80], "leverage": [0.5, 1.0, 1.5, 2.0]},
    {"lookback": [5, 10, 20, 40, 80], "smooth_span": [1, 5, 10, 20, 40]},
]

grid_results = grid_search(
    generate_weights=generate_weights,
    objective_metric=Metrics.SHARPE,
    maximize=True,
    base_params=base_params,
    param_grids=search_sets,
    progress=True,
    market=MarketData(
        close_prices=close_prices,
        order_prices=open_prices.shift(-1),  # next-period open prices
        funding_rates=funding_rates,
    ),
    config=SimConfig(
        init_cash=10_000.0,
        fee_rate=0.0002,
        slippage_rate=0.001,
        order_notional_min=10.0,
        freq_rule="1D",
        trading_days_year=365,
        risk_free_rate=0.003,
    ),
    max_workers=8,
)

grid_results.objective_metric


In [ ]:
grid_results.table.head()

## Generate a tearsheet from the best result

In [ ]:
html_str = tearsheet(grid_results=grid_results, output_path="./search_tearsheet.html")
display(HTML(html_str))